## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-20-52-07 +0000,nypost,News of the World: What you missed this week i...,https://nypost.com/2025/12/06/world-news/news-...
1,2025-12-06-20-50-16 +0000,nypost,Not groovy: ‘Disco ball’ causing inferno on UW...,https://nypost.com/2025/12/06/us-news/not-groo...
2,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...
3,2025-12-06-20-47-00 +0000,wsj,China’s Growth Is Coming at the Rest of the Wo...,https://www.wsj.com/economy/trade/chinas-growt...
4,2025-12-06-20-46-31 +0000,bbc,Olympic flame arrives in Rome ahead of Winter ...,https://www.bbc.com/news/videos/czdg0598ry0o?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,36
540,netflix,15
542,warner,14
109,new,12
543,bros,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,74
118,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...,70
208,2025-12-05-21-13-25 +0000,nyt,Gianni Infantino Awards Trump a FIFA Peace Pri...,https://www.nytimes.com/2025/12/05/us/politics...,66
191,2025-12-05-21-59-00 +0000,wsj,Netflix’s Megadeal Will Need the Trump Adminis...,https://www.wsj.com/business/media/netflix-lan...,66
22,2025-12-06-19-04-27 +0000,wapo,Trump urges a new vaccine schedule. Here’s wha...,https://www.washingtonpost.com/world/2025/12/0...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,74,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...
201,53,2025-12-05-21-43-02 +0000,nypost,How Warner Bros. Discovery’s CEO decided to se...,https://nypost.com/2025/12/05/media/how-warner...
120,45,2025-12-06-04-27-00 +0000,wsj,A key vaccine panel at the Centers for Disease...,https://www.wsj.com/politics/policy/cdc-panel-...
190,45,2025-12-05-21-59-27 +0000,wapo,Olivia Nuzzi will leave Vanity Fair amid new a...,https://www.washingtonpost.com/business/2025/1...
149,36,2025-12-06-00-22-01 +0000,nyt,San Diego Agrees to Pay $30 Million to Family ...,https://www.nytimes.com/2025/12/05/us/konoa-wi...
72,35,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
142,35,2025-12-06-01-02-00 +0000,wsj,Two survivors of a Sept. 2 lethal U.S. strike ...,https://www.wsj.com/politics/national-security...
118,34,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...
88,33,2025-12-06-11-45-00 +0000,nypost,Russia unleashes massive drone and missile att...,https://nypost.com/2025/12/06/world-news/russi...
45,32,2025-12-06-16-40-13 +0000,nyt,China’s National Security Office in Hong Kong ...,https://www.nytimes.com/2025/12/06/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
